In [7]:
import numpy as np
import itertools as it
from PGM_PyLib.MRF import RMRFwO as mrf
import requests
import pandas as pd

In [8]:
# Function to generate random probability distributions for bounding boxes
def generate_random_probabilities(num_boxes, num_states):
    """
    Generate random probability distributions for each bounding box.
    
    Parameters:
    - num_boxes: int, number of bounding boxes
    - num_states: int, number of states (e.g., "tree", "glass", "car", "cup")

    Returns:
    - numpy array of shape (num_boxes, num_states), where each row is a
      probability distribution over the states for a bounding box.
    """
    probabilities = np.random.rand(num_boxes, num_states)  # Generate random values
    probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return probabilities

# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(n):
    """Generate an n x n symmetric matrix for 0-based indexing."""
    mat = np.random.rand(n, n)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

# Define psi and Uf functions
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix (0-based indexing).
    """
    return relational_probabilities[state_i, state_j]

# Assuming relational_probabilities is global
def Uf(rmrf, observation, row, col):
    """
    Local energy function for a bounding box in a complete graph (3 rows, 1 column RMRF).
    
    This function calculates the energy based on the current state of the bounding box
    and its relationships with all other bounding boxes (complete graph assumption).
    Additionally, it considers n-tuples (n >= 3) by averaging the weights for all such tuples.
    Observation probabilities are also factored into the energy calculation.
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (3 rows, 1 column, states should be 0-based)
    - observation: bounding box probability distributions
    - row: int, row index (0-based, corresponding to the bounding box)
    - col: int, column index (should always be 0 since it's a 1-column matrix)

    Returns:
    - float, the calculated energy for the bounding box at row `row`
    """
    current_state = rmrf[row, col]  # Get the current state of the bounding box
    energy = 0.0
    
    # 1. Factor in the observation (1-tuple) likelihood for the bounding box
    obs_prob = observation[row][current_state]  # Probability for the current state
    energy -= np.log(obs_prob)  # Add the energy contribution from the observation
    
    # 2. Calculate the pairwise (2-tuple) energy from the relationships with other bounding boxes (complete graph)
    num_boxes = rmrf.shape[0]  # Number of bounding boxes (rows in rmrf)

    for other_row in range(num_boxes):
        if other_row != row:  # Avoid self-relations
            other_state = rmrf[other_row, col]  # Get the state of the other bounding box
            # Get the potential (weight) from the relational probability matrix
            weight = relational_probabilities[current_state, other_state]
            energy -= np.log(weight)  # Add the energy contribution from the relationship

    # 3. Consider n-tuples (n >= 3) and calculate the average weight as basis for the energy
    # Iterate over all possible n-tuples (where n >= 3)
    for n in range(3, num_boxes + 1):  # Start from n=3, up to n=num_boxes
        # Get all combinations of bounding boxes of size n
        tuples = list(it.combinations(range(num_boxes), n))
        
        # For each n-tuple, calculate the average relational probability
        for t in tuples:
            # Extract the relational probabilities for all pairs in the tuple
            pairwise_weights = [
                relational_probabilities[rmrf[i, col], rmrf[j, col]]
                for i, j in it.combinations(t, 2)
            ]
            
            # Calculate the mean of the pairwise weights using NumPy
            if pairwise_weights:  # Ensure there are weights to average
                average_weight = np.mean(pairwise_weights)
                energy -= np.log(average_weight)  # Subtract the energy from the average weight
    
    return energy

In [9]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
relational_probabilities = generate_symmetric_matrix(4)
print("\nRandom Relational Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
bounding_box_probabilities = generate_random_probabilities(num_boxes=3, num_states=4)
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(bounding_box_probabilities, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, bounding_box_probabilities)  # Use the bounding box probabilities as the observation

# Run inference using the Uf function
result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")

print("\nFinal RMRF after inference (Bounding Box States):")
print(result)  # This will be a 3x1 matrix with the final state for each bounding box

Initial RMRF
 [[0]
 [0]
 [0]]

Random Relational Probability Matrix (ConceptNet):
[[1.         0.44599629 0.29371489 0.18299106]
 [0.44599629 1.         0.61017296 0.63973303]
 [0.29371489 0.61017296 1.         0.46019551]
 [0.18299106 0.63973303 0.46019551 1.        ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.47163789 0.08513666 0.23071299 0.21251245]
 [0.07337819 0.52592854 0.30065232 0.10004095]
 [0.0985744  0.27395922 0.28960655 0.33785983]] 

Succesfully finish, iteration: 0

Final RMRF after inference (Bounding Box States):
[[0]
 [0]
 [0]]


In [10]:
# Define the search endpoint for fetching edges with the 'AtLocation' relation
search_url = 'http://api.conceptnet.io/query'

# Function to search for edges with 'AtLocation' relation, including 'glass' as both start and end
def get_atlocation_edges_both_directions(concept, max_limit=50):
    params_start = {
        'start': f'/c/en/{concept}',  # Fetch edges where 'glass' is the start
        'rel': '/r/AtLocation',
        'limit': max_limit
    }
    params_end = {
        'end': f'/c/en/{concept}',    # Fetch edges where 'glass' is the end
        'rel': '/r/AtLocation',
        'limit': max_limit
    }

    # Fetch edges where 'glass' is the start
    response_start = requests.get(search_url, params=params_start).json()
    edges_start = response_start['edges']

    # Fetch edges where 'glass' is the end
    response_end = requests.get(search_url, params=params_end).json()
    edges_end = response_end['edges']

    # Combine both sets of edges
    return edges_start + edges_end

# Fetch 'AtLocation' edges for the concept "glass" with a limit
atlocation_edges = get_atlocation_edges_both_directions('glass', max_limit=100000)

# Print the number of 'AtLocation' edges retrieved
print(f"Total 'AtLocation' edges retrieved for 'glass': {len(atlocation_edges)}")

#Show all the weight and concepts for AtLocation of glas and show diretcion in a visual way
for edge in atlocation_edges:
    start = edge['start']['label'].split('/')[-1]
    end = edge['end']['label'].split('/')[-1]
    weight = edge['weight']
    print(f"{start} -> {end} (Weight: {weight})")

Total 'AtLocation' edges retrieved for 'glass': 17
a glass -> the cabinet (Weight: 4.0)
a glass -> a window (Weight: 3.4641016151377544)
a glass -> a kitchen (Weight: 2.82842712474619)
a glass -> the table (Weight: 2.0)
a glass -> a water cooler (Weight: 1.0)
a glass -> a dishwasher (Weight: 1.0)
a glass -> a hand (Weight: 1.0)
a glass -> a sink (Weight: 1.0)
a glass -> a cuboard (Weight: 1.0)
a glass -> your kitchen cupboard (Weight: 1.0)
a glass -> a (Weight: 1.0)
a glass -> a dining room (Weight: 1.0)
glass -> a street (Weight: 1.0)
water -> a glass (Weight: 4.0)
an overflow -> a glass (Weight: 1.0)
ice -> a glass (Weight: 1.0)
ice cubes -> a glass (Weight: 1.0)


In [11]:
# Define the list of concepts in the desired order
concepts = ["tree", "glass", "car", "cup"]

# Define the base URL for the ConceptNet relatedness API
relatedness_url = 'http://api.conceptnet.io/relatedness'

# Function to get the relatedness value between two concepts
def get_relatedness(concept1, concept2):
    params = {
        'node1': f'/c/en/{concept1}',
        'node2': f'/c/en/{concept2}'
    }
    response = requests.get(relatedness_url, params=params).json()
    return response['value']

# Initialize a 4x4 matrix to store the relatedness values
relatedness_matrix = np.ones((4, 4))  # Start with an identity matrix (relatedness of 1 for the same concepts)

# Create all concept pairs (excluding same concept pairs as they are 1.0 by default)
concept_pairs = list(it.product(range(4), repeat=2))
concept_pairs = [(i, j) for i, j in concept_pairs if i != j]

# Fetch relatedness for each pair and populate the matrix
for i, j in concept_pairs:
    relatedness_matrix[i, j] = get_relatedness(concepts[i], concepts[j])

# Rescale the relatedness values from [-1, 1] to [0, 1]
rescaled_matrix = (relatedness_matrix + 1) / 2

# Create a pandas DataFrame for better readability with the desired order
relatedness_df = pd.DataFrame(rescaled_matrix, index=concepts, columns=concepts)

# Display the rescaled 4x4 matrix
print(relatedness_df)

         tree   glass     car     cup
tree   1.0000  0.5315  0.5450  0.4920
glass  0.5315  1.0000  0.5365  0.6345
car    0.5450  0.5365  1.0000  0.5180
cup    0.4920  0.6345  0.5180  1.0000


In [17]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
# Relational probability matrix
relational_probabilities = np.array([
    [1.0, 0.5, 0.7, 0.9],
    [0.5, 1.0, 0.6, 0.8],
    [0.7, 0.6, 1.0, 0.85],
    [0.9, 0.8, 0.85, 1.0]
])

print("\nRescaled Relatedness Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
observation = np.array([
    [0.5, 0.3, 0.1, 0.1],  # Bounding Box 0
    [0.1, 0.6, 0.2, 0.1],  # Bounding Box 1
    [0.1, 0.2, 0.6, 0.1]   # Bounding Box 2
])
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(bounding_box_probabilities, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, observation)  # Use the bounding box probabilities as the observation

Initial RMRF
 [[0]
 [0]
 [0]]

Rescaled Relatedness Probability Matrix (ConceptNet):
[[1.   0.5  0.7  0.9 ]
 [0.5  1.   0.6  0.8 ]
 [0.7  0.6  1.   0.85]
 [0.9  0.8  0.85 1.  ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.47163789 0.08513666 0.23071299 0.21251245]
 [0.07337819 0.52592854 0.30065232 0.10004095]
 [0.0985744  0.27395922 0.28960655 0.33785983]] 



In [13]:
# Run the inference n times and display the results for each run and the counts for each unique result
n = 10000  # Number of inference runs
results = []

for i in range(n):
    # Create an instance of RMRFwO and run inference
    mr = mrf(s, r, observation)  # Use the bounding box probabilities as the observation
    result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")
    results.append(result)

# Count the occurrences of each unique result
unique_results, counts = np.unique(results, axis=0, return_counts=True)

# List the 3 most common results and their counts, ordered by count
sorted_results = sorted(zip(unique_results, counts), key=lambda x: x[1], reverse=True)
print("\nTop 3 most common results:")
for result, count in sorted_results[:3]:
    print(f"Result: {result.ravel()} (Count: {count})")

Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfully finish, iteration: 2
Succesfully finish, iteration: 0
Succesfully finish, iteration: 0
Succesfull

In [15]:
# Function to calculate energy for a given assignment
def calculate_energy(assignment):
    energy = 0.0
    
    # Observation contributions
    for row in range(len(assignment)):
        state = assignment[row]
        obs_prob = observation[row][state]
        energy -= np.log(obs_prob)

    # Pairwise (2-tuple) contributions
    for i, j in it.combinations(range(len(assignment)), 2):
        state_i = assignment[i]
        state_j = assignment[j]
        weight = relational_probabilities[state_i, state_j]
        energy -= np.log(weight)

    # n-tuple (n >= 3) contributions
    tuples = list(it.combinations(range(len(assignment)), 3))
    for t in tuples:
        pairwise_weights = [
            relational_probabilities[assignment[i], assignment[j]]
            for i, j in it.combinations(t, 2)
        ]
        average_weight = np.mean(pairwise_weights)
        energy -= np.log(average_weight)

    return energy

# Generate all possible state assignments for 3 bounding boxes and 4 states
all_assignments = list(it.product([0, 1, 2, 3], repeat=3))

# Calculate energy for each assignment
results = []
for assignment in all_assignments:
    energy = calculate_energy(assignment)
    results.append((assignment, energy))

# Convert results to a DataFrame for better visualization
df = pd.DataFrame(results, columns=["Assignment", "Energy"])
df["Assignment"] = df["Assignment"].apply(lambda x: str(x))  # Format the assignments for display

# Find the minimum energy assignment
min_energy = df["Energy"].min()
min_assignment = df.loc[df["Energy"].idxmin()]

# Display the DataFrame and highlight the minimum energy assignment
print(df)

print(f"Minimum energy assignment: {min_assignment['Assignment']} with energy {min_energy}")

   Assignment    Energy
0   (0, 0, 0)  5.298317
1   (0, 0, 1)  6.396930
2   (0, 0, 2)  4.443051
3   (0, 0, 3)  5.578031
4   (0, 1, 0)  5.298317
..        ...       ...
59  (3, 2, 3)  6.645006
60  (3, 3, 0)  7.187469
61  (3, 3, 1)  6.803996
62  (3, 3, 2)  5.546394
63  (3, 3, 3)  6.907755

[64 rows x 2 columns]
Minimum energy assignment: (1, 1, 1) with energy 3.324236340526027
